In [0]:
#Using dbutils.fs.ls command to check the file that been uploaded

dbutils.fs.ls("/FileStore/tables/")

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592, modificationTime=1706721501000),
 FileInfo(path='dbfs:/FileStore/tables/accounts2/', name='accounts2/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip', name='activations.zip', size=8411369, modificationTime=1706718610000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023/', name='clinicaltrial_2023/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023-1.zip', name='clinicaltrial_2023-1.zip', size=52785750, modificationTime=1712308832000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023.zip', n

Defining Reusable code to unzip all files

In [0]:
fileroot = "clinicaltrial_2023"

dbutils.fs.cp("FileStore/tables/" + fileroot + ".zip", "file:/tmp/")
#import os library

import os
os.environ['fileroot'] = fileroot


In [0]:
%sh

unzip -d /tmp /tmp/$fileroot.zip

Archive:  /tmp/clinicaltrial_2023.zip
  inflating: /tmp/clinicaltrial_2023.csv  


In [0]:
# List the contents of the /tmp/ directory
files_in_tmp = dbutils.fs.ls("file:/tmp/")

# Print the list of files and directories in /tmp/
for file_info in files_in_tmp:
    print(file_info.path)

file:/tmp/hsperfdata_root/
file:/tmp/.ICE-unix/
file:/tmp/clinicaltrial_2023.csv
file:/tmp/.X11-unix/
file:/tmp/tmp.yqG28XVDrF
file:/tmp/driver-daemon-params
file:/tmp/python_lsp_logs/
file:/tmp/systemd-private-0309b29f20584f99a3c6a0c6b5accc6b-apache2.service-99piQh/
file:/tmp/custom-spark.conf
file:/tmp/clinicaltrial_2023.zip
file:/tmp/systemd-private-0309b29f20584f99a3c6a0c6b5accc6b-systemd-resolved.service-LIqvAf/
file:/tmp/chauffeur-daemon-params
file:/tmp/chauffeur-env.sh
file:/tmp/chauffeur-daemon.pid
file:/tmp/systemd-private-0309b29f20584f99a3c6a0c6b5accc6b-systemd-logind.service-n2gd4e/
file:/tmp/.XIM-unix/
file:/tmp/RtmpBNopsC/
file:/tmp/systemd-private-0309b29f20584f99a3c6a0c6b5accc6b-ntp.service-xq2A9g/
file:/tmp/driver-daemon.pid
file:/tmp/driver-env.sh
file:/tmp/Rserv/
file:/tmp/.Test-unix/
file:/tmp/.font-unix/


In [0]:
#Make directory

dbutils.fs.mkdirs("FileStore/tables/" + fileroot)

#Move files

dbutils.fs.mv("file:/tmp/" + fileroot + ".csv" , "/FileStore/tables/" + fileroot , True)

# remove the .zip file

dbutils.fs.rm("/FileStore/tables/" + fileroot + ".zip")

dbutils.fs.ls("/FileStore/tables/" + fileroot)

Out[5]: [FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023/clinicaltrial_2023.csv', name='clinicaltrial_2023.csv', size=202413369, modificationTime=1713181363000)]

In [0]:
file = "/FileStore/tables/clinicaltrial_2023/clinicaltrial_2023.csv"

#dbutils.fs.head(file)

In [0]:
# Create an rdd

rdd1 = sc.textFile(file)

rdd1.take(3)

Out[7]: ['NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design,Start Date,Completion Date',
 '"NCT03630471 \\t Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India \\t PRIDE \\t COMPLETED \\t Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) \\t BEHAVIORAL: PRIDE \'Step 1\' problem-solving intervention|BEHAVIORAL: Enhanced usual care \\t Sangath \\t Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine \\t 250 \\t OTHER \\t INTERVENTIONAL \\t Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019',
 '"NCT05992571 \\t Oral Ketone Monoester Supplementation and Resting-state Brain Connectivity \\t RECRUITING \\t Cerebrovascular Function|Cog

In [0]:
header = rdd1.first()

colRDD = rdd1 \
        .filter(lambda x: x != header) \
        .flatMap(lambda x: x.split("\n")) \
        .filter(lambda x: x.strip() != "") \
        .map(lambda x: x.split("\\t")[:1]) \

colRDD.take(4)
df = colRDD.toDF()
#df = df.toDF("NCT Number", "Study Title","Acronym","Study Status","Conditions","Interventions","Sponsor","Collaborators")
df = df.toDF("NCT Number")

In [0]:
colRDD.take(4)

Out[9]: [['"NCT03630471 '], ['"NCT05992571 '], ['NCT01854671 '], ['NCT03820271 ']]

In [0]:
df.show(1, truncate= False)

+-------------+
|NCT Number   |
+-------------+
|"NCT03630471 |
+-------------+
only showing top 1 row



In [0]:
df.printSchema()

root
 |-- NCT Number: string (nullable = true)



In [0]:
#df = df.withColumn("Enrollment", df.Enrollment.cast("int"))

#df.printSchema()

In [0]:
from pyspark.sql.functions import col

distinct_study_count = df.select(col('NCT Number')).distinct().count()

#output
print("Number of distinct studies:", distinct_study_count)

Number of distinct studies: 478697


In [0]:
header = rdd1.first()

colRDD2 = rdd1 \
        .filter(lambda x: x != header) \
        .flatMap(lambda x: x.split("\n")) \
        .filter(lambda x: x.strip() != "") \
        .map(lambda x: x.split("\\t")[:9][-2:]) \

colRDD2.take(4)
df2 = colRDD2.toDF()
#df = df.toDF("NCT Number", "Study Title","Acronym","Study Status","Conditions","Interventions","Sponsor","Collaborators")

In [0]:
from pyspark.sql.functions import explode, split, col, count

exploded_df_1 = df2.select(explode(split(col("_1"),"\|")).alias("word_1"))
exploded_df_2 = df2.select(explode(split(col("_2"),"\|")).alias("word_2"))

exploded_df = exploded_df_1.union(exploded_df_2)

exploded_df.show(5)

+--------------------+
|              word_1|
+--------------------+
| Harvard Medical ...|
|London School of ...|
|                 30 |
|                176 |
|                500 |
|      OBSERVATIONAL |
|           INDUSTRY |
|           INDUSTRY |
|      OBSERVATIONAL |
|                 42 |
|                 10 |
| King's College H...|
|Charles Universit...|
|Hopital Universit...|
|Vestre Viken Hosp...|
|The Hospital for ...|
|  Cardiff University|
|Odense University...|
|     INTERVENTIONAL |
|              OTHER |
| National Natural...|
|           INDUSTRY |
|                 30 |
| Centre de Recher...|
|              OTHER |
|           INDUSTRY |
|                 50 |
|                 62 |
|              OTHER |
|                 90 |
|              OTHER |
|                 85 |
|              OTHER |
|              OTHER |
|      OBSERVATIONAL |
|      OBSERVATIONAL |
|              OTHER |
|                 44 |
|                  0 |
|                 30 |
|          